In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### LSV

In [3]:
query = '''
    DECLARE @RC int
    DECLARE @TimeFrame int
    DECLARE @Investor int
    DECLARE @CreditRisk int

    EXECUTE @RC = [dbo].[Measures_Lsv_Bionomial] 
        {},
        {},
        {}
'''

credit = {
    'NULL': '', 1: '-hy', 2: '-ig'
}
investor = {
    2: '-retail', 3: '-institutional'
}
timeframe = {
    1: '-daily', 2: '-weekly', 3: '-monthly', 4: '-quarterly', 5: '-yearly'
}

for tf, tf_desc in timeframe.items():
    for inv, inv_desc in investor.items():
        for crd, crd_desc in credit.items():
            df = pd.read_sql_query(query.format(tf, inv, crd), connection)
            df.to_csv('source/lsv{}{}{}.csv'.format(inv_desc, crd_desc, tf_desc), index=False)